In [1]:
'''
Fine tuning of the models from Nassano' thesis.
The overall trained models are used (conditional + fine tuning from the thesis).
The plan is to produce both fine tuned and reinforcement learned (si dice così?) models, starting with
the formers for the sake of celerity. The training pipeline is taken from his github repo and his notebook
'''

"\nFine tuning of the models from Nassano' thesis.\nThe overall trained models are used (conditional + fine tuning from the thesis).\nThe plan is to produce both fine tuned and reinforcement learned (si dice così?) models, starting with\nthe formers for the sake of celerity. The training pipeline is taken from his github repo and his notebook\n"

In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
# from tensorflow.python.compiler.mlcompute import mlcompute
# mlcompute.set_mlc_device(device_name='gpu')
#
# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()

import pathlib
from IPython import display
import functools
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import time
from tqdm import tqdm
# from mitdeeplearning import as mdl
import cv2 as cv
import funcs as ff
import os
from VAE import VAE
from datetime import datetime

# check that the environment is fine
print('TensorFlow version : ', tf.__version__ )
print('NumPy version : ', np.__version__ )
print('OpenCV version', cv.__version__)

print('gpu list : ', tf.config.list_physical_devices())

/opt/homebrew/Caskroom/miniforge/base/envs/tf-arm/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.0-rc0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


TensorFlow version :  2.4.0-rc0
NumPy version :  1.20.1
OpenCV version 4.5.1
gpu list :  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [3]:

'''
DIRECTORY SETTINGS
'''
cwd = '/Volumes/Seagate Backup Plus Drive/EXPERIMENTS' #os.getcwd()

serie_dir = os.path.join(cwd, 'Embeddings')
if not os.path.isdir(serie_dir):
    os.makedirs(serie_dir)

exp_dir = os.path.join(serie_dir, 'Xception-vae-finetuning')
if not os.path.isdir(exp_dir):
    os.makedirs(exp_dir)

now = datetime.now().strftime('%b%d-%Y')
print(now)
date_dir = os.path.join(exp_dir, now)
if not os.path.isdir(date_dir):
    os.makedirs(date_dir)

last_ckpt = os.path.join(exp_dir, 'last_ckpt')
if not os.path.isdir(last_ckpt):
  os.mkdir(last_ckpt)

print('cwd : ', cwd)
print('serie dir : ', serie_dir)
print('exp_dir : ', exp_dir)
print('date_ dir : ', date_dir)
print('last ckpt dir : ', last_ckpt)


Apr22-2021
cwd :  /Volumes/Seagate Backup Plus Drive/EXPERIMENTS
serie dir :  /Volumes/Seagate Backup Plus Drive/EXPERIMENTS/Embeddings
exp_dir :  /Volumes/Seagate Backup Plus Drive/EXPERIMENTS/Embeddings/Xception-vae-finetuning
date_ dir :  /Volumes/Seagate Backup Plus Drive/EXPERIMENTS/Embeddings/Xception-vae-finetuning/Apr22-2021
last ckpt dir :  /Volumes/Seagate Backup Plus Drive/EXPERIMENTS/Embeddings/Xception-vae-finetuning/last_ckpt


In [4]:

def record_parser(example):
    example_fmt = {
        'label': tf.io.FixedLenFeature([14], tf.float32),
        'image': tf.io.FixedLenFeature([],tf.string, default_value='')}
    parsed = tf.io.parse_single_example(example, example_fmt)
    image = tf.io.decode_png(parsed["image"],channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, parsed['label']

def normalize_image(img,labels):
    imagenet_mean = np.array([0.485, 0.456, 0.406])
    imagenet_std = np.array([0.229, 0.224, 0.225])
    img = (img - imagenet_mean) / imagenet_std
    return img,labels

def make_dataset(filename):
    base_path = '/Volumes/Seagate Backup Plus Drive/datasets/chexpert-tfrecords'
    full_path = os.path.join(base_path,filename)
    dataset = tf.data.TFRecordDataset(full_path)
    parsed_dataset = dataset.map(record_parser,num_parallel_calls = tf.data.experimental.AUTOTUNE)
    parsed_dataset = parsed_dataset.map(normalize_image,num_parallel_calls = tf.data.experimental.AUTOTUNE)
    return parsed_dataset

batch_size = 5

train_ds = make_dataset('new_training_cropped.tfrecords').shuffle(buffer_size=128).batch(batch_size, drop_remainder=True).prefetch(1)
val_ds = make_dataset('validation_cropped.tfrecords').batch(batch_size, drop_remainder=True).prefetch(1)
test_ds = make_dataset('test_set_cropped.tfrecords').batch(batch_size, drop_remainder=False).prefetch(1)
label_names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
                'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [5]:

# ###############
# Model
# ###############
from tensorflow.keras.models import load_model
filepath = '/Volumes/Seagate Backup Plus Drive/models/ChestXRay/ConditionalTraining/Xception/FineTuning'
model_name = 'model-05.hdf5'
model = load_model(filepath=os.path.join(filepath, model_name))

vae = VAE(dummy_model=False, latent_dim=100, number_of_classes=0)
vae.load_encoder(model=model, keep_last_layer=False)
vae.load_decoder(model=ff.make_std_decoder(conv_steps=5, final_images_size=(224,224)))
#vae.encoder.load_weights(os.path.join(last_ckpt, 'encoder-last'))
#vae.encoder.load_weights('/Volumes/Seagate Backup Plus Drive/models/ChestXRay/ConditionalTraining/Xception/FineTuning/model-05.hdf5')
vae.decoder.load_weights(os.path.join(last_ckpt, 'decoder-last'))
#vae.decoder.load_weights('/Volumes/Seagate Backup Plus Drive/EXPERIMENTS/Embeddings/Xception-vae/last_ckpt/decoder-last')
vae.summary(expected_dims=[224, 224, 3])

self.latent_dim : 100
self.num_classes : 0
self.bottleneck : 200
decoder initial dim :  7
Encoder : 
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]      

In [6]:
'''
# ###############
# TRAINING LOOP FUNCTIONS
# ###############
'''
lr = .5e-4
epochs = 200
optimizer = tf.keras.optimizers.Adam(lr)

''' Pick a sample of the test set for generating output images
'''
num_examples_to_generate = 4
test_sample = None
assert batch_size >= num_examples_to_generate
for test_batch, test_labels in val_ds.take(1):
  test_sample = test_batch[0:num_examples_to_generate, :, :, :]


def tmp_plot(model, test_sample, loss, metric = None):
    _, _, x_recon, _ = model(test_sample)
    x_recon = ff.normalize_tensor(x_recon)
    test_sample = ff.normalize_tensor(test_sample)
    fig = plt.figure(figsize=(20, 15))
    for i in range(x_recon.shape[0]):
        plt.subplot(1, 4, i + 1)
        plt.imshow(x_recon[i, :, :, :])
        plt.axis('off')
    fig = plt.figure(figsize=(20, 15))
    for i in range(test_sample.shape[0]):
        plt.subplot(1, 4, i + 1)
        plt.imshow(test_sample[i, :, :, :])
        plt.axis('off')
    fig = plt.figure(figsize=(20, 10))
    plt.title('Loss')
    plt.semilogy(range(loss.shape[0]), loss.numpy())
    if metric is not None:
        fig = plt.figure(figsize=(20, 10))
        plt.title('Hamming distance')
        plt.plot(range(metric.shape[0]), metric.numpy())
    display.clear_output(wait=False)
    plt.show()

# Training step with gradient tapes
@tf.function
def vae_train_step(x, optimizer):
  with tf.GradientTape() as tape:
    # Feed input x into dbvae. Note that this is using the DB_VAE call function!
    z_mu, z_log_sigma, x_rec, y_hat = vae(x)
    loss = VAE.vae_loss_function(x, x_rec, z_mu, z_log_sigma)
  grads = tape.gradient(loss, vae.trainable_variables)
  # apply gradients to variables
  optimizer.apply_gradients(zip(grads, vae.trainable_variables))
  return loss, y_hat


In [ ]:
'''TRAINING LOOP'''


helper = ff.TrainingUtility(
    model_list=[vae.encoder, vae.decoder],
    model_names_list=['encoder', 'decoder']
)

for epoch in tqdm(range(epochs)):
    losses = tf.zeros([1], dtype=tf.float32)
    count = 0
    start_time = time.time()
    for train_x, train_labels in tqdm(train_ds):
        train_loss, logits = vae_train_step(train_x, optimizer)
        losses = tf.concat([losses, tf.expand_dims(tf.math.reduce_mean(train_loss), axis=0)], axis=0)
        count += 1
        if not bool(count%5):
            count = 0
            helper.save_checkpoints(path=last_ckpt, suffix='-last')
            tmp_plot(vae, test_sample, losses)
    end_time = time.time()
    train_time = end_time - start_time

    start_time = time.time()
    losses = tf.zeros([1], dtype=tf.float32)
    for test_x, test_labels in val_ds:
        z_mean, z_logsigma, x_recon, y_logits = vae(test_x)
        vae_val_loss = (VAE.vae_loss_function(test_x, x_recon, z_mean, z_logsigma))
        losses = tf.concat([losses, tf.expand_dims(tf.reduce_mean(vae_val_loss), axis=-1)], axis=0)
    valid_loss = tf.reduce_mean(losses[1:, ...])
    end_time = time.time()
    val_time = end_time - start_time

    helper.add_loss(train_value=train_loss, val_value=valid_loss)
    helper.plot_loss()
    helper.plot_imgs(imgs=x_recon[:4], imgs2=test_x[:4])
    helper.show_plots()
    helper.earlystopping_loss()
    helper.earlystopping_overfitting()
    helper.save_checkpoints(path=date_dir)
    lr=lr/10
    print('Epoch: {}'
          '\nTraining set Loss : {}'
          '\nValidation set Loss: {}'
          '\ntime elapse for current epoch: training {} validation {}'
        .format(epoch, train_loss, valid_loss, train_time, val_time))

  0%|          | 0/200 [00:00<?, ?it/s]
0it [00:00, ?it/s]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
